In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pathlib import Path
len(list((Path.home() / 'data/e-delib/kialo/kialoV2').glob('**/*.pkl')))

In [ ]:
len(list((Path.home() / 'data/e-delib/kialo/kialoV2/english').glob('**/*.pkl')))

In [ ]:
len(list((Path.home() / 'data/e-delib/kialo/kialoV2/other').glob('**/*.pkl')))

In [ ]:
import pandas as pd
from pathlib import Path

from kialo_util import read_data

maps = read_data({'local': True, 'debug_maps_size': 10, 'debug_maps_size': None})

In [ ]:
maps[0].name

In [ ]:
import pandas as pd

maps_df = pd.DataFrame([{'id': m.id, 'size': m.number_of_children(), 'name': m.name} for m in maps]).set_index('id')
maps_df

In [ ]:
experiment = 'kialov2-no_hard_negatives-detailed'

from pathlib import Path
import json
    
data_path = Path.home() / 'data/delib-results/results'
results_path = data_path / experiment / '-results'

# with open(results_path / 'all/all_maps.json') as data_file:    
#     data = json.load(data_file)  
# df = pd.DataFrame.from_dict(data, orient='index')
# df = pd.json_normalize(df)
# df
    
df = pd.read_json(results_path / 'all/all_maps.json', orient='index')
# display(df)
# df.iloc[0]['all']['mrr']
df_dict = {k: pd.json_normalize(df[k].apply(lambda x: {} if pd.isna(x) else x)).set_index(df.index) for k in df.columns}
df_dict['all']

In [ ]:
df_dict.keys()

In [ ]:
results_df = df_dict['only_leafs_limited_types'].join(maps_df)
results_df

In [ ]:
results_df[results_df['size']<=20]

In [ ]:
for s in [20, 50, 100, 500, 1000, 2000]:
    print(f"<{s:<10}{results_df[results_df['size']<s]['p5'].mean():.4f}")
print(f">={s:<10}{results_df[results_df['size']>=s]['p5'].mean():.4f}")

In [ ]:
results_df.hist('size');

In [ ]:
results_df.plot(x='size', y='p5', kind='scatter');

In [ ]:
import seaborn as sns
# sns.lmplot(x='size', y='p5', data=results_df)
# sns.kdeplot(y='size', x='p5', data=results_df)
# sns.kdeplot(results_df[['size', 'p5']])